In [6]:
pip install translate-api

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 79.8/79.8 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 52.8/52.8 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 4.0 MB/s eta 0:00:00
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14598 sha256=762c7983480dea8167e3819b1e975fec93a5fd44bc661d5438b9f2457df2be16
  Stored in directory: c:\users\maxime.alter\appdata\local\pip\cache\wheels\02\68\54\e702279a4359428d51796ed57b9be98018f60d3981361023c3
Successfully built PyExecJS
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import numpy as np
import os
from pathlib import Path 
import missingno as msno
from tabulate import tabulate
from statistics import median, mean, quantiles
import pprint
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import geopy.distance
import folium
import geopy.distance
import translators as ts
plt.style.use('ggplot')
#pd.set_option('max_columns', 200)

Using France server backend.


In [2]:
# Création d'un DF pour afficher les stats des données
def tstats (data) :
    output = []
    for col in data.columns:
    
        nonNull  = len(data) - np.sum(pd.isna(data[col]))
        nonNullprop = (nonNull / len(data[col]))*100
        unique = data[col].nunique()
        colType = str(data[col].dtype)
        output.append([col, colType, nonNull, round(nonNullprop, 1) , unique])
            
    
    df_stats = pd.DataFrame(output)
    df_stats.columns = ['nom colonne','dtype', 'valeur non null',"% de non null", 'nb_unique']

    print(tabulate(df_stats, headers='keys', tablefmt='psql'))
    
    
def diff_in_hours (date1, date2) :
    result_hour = (date1 - date2) / pd.Timedelta('1 hour')
    return result_hour


def filter_geo_brazil_lat(latitudes):
    filtered_latitudes = []
    default_lat = sum(latitudes) / len(latitudes)
    for lat in latitudes:
         if -33.75 <= lat <= 5.5:
            filtered_latitudes.append(lat)
         else : 
            filtered_latitudes.append(default_lat)            
    return filtered_latitudes


def filter_geo_brazil_lng(longitudes):
    filtered_longitudes = []
    default_lng = sum(longitudes) / len(longitudes)
    for lon in longitudes:
        if -74.5 <= lon <= -34.5:
            filtered_longitudes.append(lon)
        else : 
            filtered_longitudes.append(default_lng)
    return filtered_longitudes

In [4]:
path = Path(os.getcwd()) 
p_parent = path.parent
p_customer = str(p_parent) + '\data\olist_customers_dataset.csv'
p_geo = str(p_parent) + '\data\olist_geolocation_dataset.csv'
p_order_item = str(p_parent) + '\data\olist_order_items_dataset.csv'
p_order_payment = str(p_parent) + '\data\olist_order_payments_dataset.csv'
p_order_review = str(p_parent) + '\data\olist_order_reviews_dataset.csv'
p_orders = str(p_parent) + '\data\olist_orders_dataset.csv'
p_products = str(p_parent) + '\data\olist_products_dataset.csv'
p_sellers = str(p_parent) + '\data\olist_sellers_dataset.csv'
p_category = str(p_parent) + '\data\product_category_name_translation_expanded.csv'

customer = pd.read_csv(p_customer)
geo = pd.read_csv(p_geo)
order_item = pd.read_csv(p_order_item)
order_payment = pd.read_csv(p_order_payment)
order_review = pd.read_csv(p_order_review)
orders = pd.read_csv(p_orders)
products = pd.read_csv(p_products)
sellers = pd.read_csv(p_sellers)
category = pd.read_csv(p_category)


#fusion avec .merge

df = orders.merge(customer, on="customer_id", how ='left')\
.merge(order_item, on= "order_id", how = 'left')\
.merge(order_payment, on ='order_id', how ='left')\
.merge(order_review, on ="order_id", how="left")\
.merge(sellers, on ='seller_id', how ="left")\
.merge(products, on = "product_id", how = 'left')\
.merge(category, on="product_category_name", how="left")

In [5]:
# on va créer un autre df pour traiter ces données :

# création de la base des vendeurs :
df_geo_seller = sellers.merge(geo, left_on ="seller_zip_code_prefix", right_on ='geolocation_zip_code_prefix', how ='inner')
#filtre des données hors brezil : 
df_geo_seller['geolocation_lat'] = filter_geo_brazil_lat(df_geo_seller['geolocation_lat'])
df_geo_seller['geolocation_lng'] = filter_geo_brazil_lng(df_geo_seller['geolocation_lng'])
#regroupement des données par vendeur : 
gr_geo_seller = df_geo_seller.groupby(['seller_id'], as_index=False).aggregate({
                'geolocation_lat':'mean',
                'geolocation_lng' : 'mean',
                }).rename(columns={"geolocation_lat": "seller_lat",
                            "geolocation_lng": "seller_lng",
                }) 

# création de la base des clients :
df_geo_customer = customer.merge(geo, left_on ="customer_zip_code_prefix", right_on ='geolocation_zip_code_prefix', how ='inner')
#filtre des données hors brezil :  
df_geo_customer['geolocation_lat'] = filter_geo_brazil_lat(df_geo_customer['geolocation_lat'])
df_geo_customer['geolocation_lng'] = filter_geo_brazil_lng(df_geo_customer['geolocation_lng'])
#regroupement des données par client unique : 
gr_geo_customer = df_geo_customer.groupby(['customer_unique_id'], as_index=False).aggregate({
                'geolocation_lat':'mean',
                'geolocation_lng' : 'mean',
                }).rename(columns={"geolocation_lat": "customer_lat",
                            "geolocation_lng": "customer_lng",
                })


df = df.merge(gr_geo_seller, left_on ="seller_id", right_on ='seller_id', how ='inner')\
.merge(gr_geo_customer, left_on ="customer_unique_id", right_on ='customer_unique_id', how ='inner')

In [9]:
#création d'un liste des nom de colonnes
cols = []

for col in df : 
    cols.append(col)

print(cols)

['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value', 'review_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp', 'seller_zip_code_prefix', 'seller_city', 'seller_state', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'product_category_name_english', 'metacategory', 'seller_lat', 'seller_lng', 'customer_lat', 'customer_lng']


In [25]:
##translate module : cheack https://pypi.org/project/translate-api/

#for label, row in df.iterrows() : 
#    if pd.notna(row['review_comment_title']) | pd.notna(row['review_comment_message']):
#        title_trad = ts.google(str(row['review_comment_title']))
#        message_trad =  ts.google(row['review_comment_message'])
#        df.loc [label, "titre_trad"] = title_trad
#        df.loc [label, "message_trad"] = message_trad
#
#print(df)